<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/city_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 行政情報のexcelファイルををjsonファイルに変換して出力

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import json

In [3]:
DIR_NAME = '/content/drive/MyDrive/z-gard/行政情報'

In [4]:
df_city_info = pd.read_excel(os.path.join(DIR_NAME, '23区行政サービス一覧_入力テンプレート220130.xlsx'), sheet_name=1, header=1, index_col=None)

df_city_info = df_city_info.rename(columns={
    '項目': 'title',
    '区の公式HPの情報URL': 'URL',
    '区の公式HPに掲載の補助金・手当名': 'name',
    '区の公式HPに掲載の補助金-詳細': 'detail'
}).drop(columns=['URL'])
print(df_city_info.shape)

(108, 4)


In [5]:
df_city_info['title'] = df_city_info['title'].replace('[①②③④]', '', regex=True)
df_city_info['name'] = df_city_info['name'].fillna('').replace('なし', '').replace('公立学校の学校選択制', '')
df_city_info['detail'] = df_city_info['detail'].fillna('').replace('', 'なし')

In [6]:
df_city_info

,区名,title,name,detail
0,千代田区,妊娠・出産関連の手当,誕生準備手当,"一妊娠につき、一時金として 45,000円を支給（多児妊娠・出産でも、同額。流産または死産の..."
1,千代田区,公立学校の学校選択制,,中学校
2,千代田区,居住関連の手当,次世代育成住宅助成,千代田区内の民間賃貸住宅またはマイホームへの住み替えをする世帯のうち、以下の1、2のいずれか...
3,千代田区,居住関連の手当,居住安定支援家賃助成,千代田区内に引き続き2年以上居住する高齢者世帯・障害者世帯・ひとり親世帯が、取り壊し等により...
4,千代田区,居住関連の手当,高齢者等民間賃貸住宅入居支援制度,千代田区内に居住する高齢者・障害者・ひとり親世帯のいずれかが区内での転居をする際に、保証会社...
...,...,...,...,...
103,葛飾区,その他独自の取り組み,,なし
104,江戸川区,妊娠・出産関連の手当,,なし
105,江戸川区,公立学校の学校選択制,,なし
106,江戸川区,居住関連の手当,,なし


### dict形式に変換して、区ごとにまとめる

In [7]:
title_list = list(df_city_info['title'].unique())
print(title_list)

['妊娠・出産関連の手当', '公立学校の学校選択制', '居住関連の手当', 'その他独自の取り組み']


In [8]:
city_list = list(df_city_info['区名'].unique())
output_dict = {city:{'info':[]} for city in city_list}

In [9]:
for city in output_dict.keys():
    for title in title_list:
        output_dict[city]['info'].append({'title': title, 'items':[]})

In [10]:
df_city_dict = df_city_info.to_dict(orient='index')
print(len(df_city_dict))

for val in df_city_dict.values():
    titles = output_dict[val['区名']]
    for t in titles['info']:
        if t['title'] == val['title']:
            t['items'].append({'name': val['name'], 'detail': val['detail']})
            break

108


### キャラクター

In [11]:
city_list

['千代田区',
 '港区',
 '新宿区',
 '文京区',
 '台東区',
 '中央区',
 '墨田区',
 '目黒区',
 '大田区',
 '江東区',
 '品川区',
 '世田谷区',
 '中野区',
 '豊島区',
 '杉並区',
 '渋谷区',
 '北区',
 '荒川区',
 '板橋区',
 '練馬区',
 '足立区',
 '葛飾区',
 '江戸川区']

In [12]:
chara_list = {
    '千代田区': ['chiyoda.jpg', '©千代田区(仮)'],
    '港区': ['minato.jpg', '©港区(仮)'],
    '新宿区': ['shinjuku.jpg', '©新宿区(仮)'],
    '文京区': ['bunkyo.jpg', '©文京区(仮)'],
    '台東区': ['taito.jpg', '©台東区(仮)'],
    '中央区': ['chuou.jpg', '©中央区(仮)'],
    '墨田区': ['sumida.jpg', '©墨田区(仮)'],
    '目黒区': ['meguro.jpg', '©目黒区(仮)'],
    '大田区': ['oota.jpg', '©大田区(仮)'],
    '江東区': ['koutou.jpg', '©江東区(仮)'],
    '品川区': ['shinagawa.jpg', '©品川区(仮)'],
    '世田谷区': ['setagaya.jpg', '©世田谷区(仮)'],
    '中野区': ['nakano.jpg', '©中野区(仮)'],
    '豊島区': ['toshima.jpg', '©豊島区(仮)'],
    '杉並区': ['suginami.jpg', '©杉並区(仮)'],
    '渋谷区': ['shibuya.jpg', '©渋谷区(仮)'],
    '北区': ['kita.jpg', '©北区(仮)'],
    '荒川区': ['arakawa.jpg', '©荒川区(仮)'],
    '板橋区': ['itahashi.jpg', '©板橋区(仮)'],
    '練馬区': ['nerima.jpg', '©2011練馬区ねり丸＃N40'],
    '足立区': ['adachi.jpg', '©足立区(仮)'],
    '葛飾区': ['katsushika.jpg', '©葛飾区(仮)'],
    '江戸川区': ['edogawa.jpg', '©江戸川区(仮)'],
}

In [13]:
for key, val in output_dict.items():
    val['character'] = chara_list[key][0]
    val['copyright'] = chara_list[key][1]

In [14]:
output_dict['千代田区']

{'info': [{'title': '妊娠・出産関連の手当',
   'items': [{'name': '誕生準備手当',
     'detail': '一妊娠につき、一時金として 45,000円を支給（多児妊娠・出産でも、同額。流産または死産の場合でも支給）。'}]},
  {'title': '公立学校の学校選択制', 'items': [{'name': '', 'detail': '中学校'}]},
  {'title': '居住関連の手当',
   'items': [{'name': '次世代育成住宅助成',
     'detail': '千代田区内の民間賃貸住宅またはマイホームへの住み替えをする世帯のうち、以下の1、2のいずれかに該当する世帯が対象。1 親元近居助成（区内に引き続き5年以上居住する親がいる新婚世帯）2 区内転居助成（区内に引き続き1年以上居住している子育て世帯）'},
    {'name': '居住安定支援家賃助成',
     'detail': '千代田区内に引き続き2年以上居住する高齢者世帯・障害者世帯・ひとり親世帯が、取り壊し等により転居を余儀なくされた場合や、やむを得ない事由により世帯の所得が著しく減少した場合、または安全上・衛生上劣悪な状態の住宅に居住している場合に、家賃等の一部を助成'},
    {'name': '高齢者等民間賃貸住宅入居支援制度',
     'detail': '千代田区内に居住する高齢者・障害者・ひとり親世帯のいずれかが区内での転居をする際に、保証会社利用料への助成等を行う'},
    {'name': '高齢者等安心居住支援家賃助成制度',
     'detail': '千代田区内の持ち家に居住する方で、その居住する住宅のバリアフリー改修が困難等の理由により居宅での日常生活に支障が生じ、緊急に代替となる住宅の確保が必要と認められる要介護高齢者世帯等に対して、民間賃貸住宅の家賃等の一部を助成（月額5万円、２年間）'}]},
  {'title': 'その他独自の取り組み', 'items': [{'name': '', 'detail': 'なし'}]}],
 'character': 'chiyoda.jpg',
 'copyright': '©千代田区(仮)'}

### jsonファイルに出力

In [15]:
output_file = os.path.join(DIR_NAME, 'city_info.json')
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_dict, f, indent=4, ensure_ascii=False)

print(output_file)

/content/drive/MyDrive/z-gard/行政情報/city_info.json


### jsonファイルを読み出して確認

In [16]:
with open(output_file) as f:
    input_json = json.load(f)

In [17]:
input_json['千代田区']

{'info': [{'title': '妊娠・出産関連の手当',
   'items': [{'name': '誕生準備手当',
     'detail': '一妊娠につき、一時金として 45,000円を支給（多児妊娠・出産でも、同額。流産または死産の場合でも支給）。'}]},
  {'title': '公立学校の学校選択制', 'items': [{'name': '', 'detail': '中学校'}]},
  {'title': '居住関連の手当',
   'items': [{'name': '次世代育成住宅助成',
     'detail': '千代田区内の民間賃貸住宅またはマイホームへの住み替えをする世帯のうち、以下の1、2のいずれかに該当する世帯が対象。1 親元近居助成（区内に引き続き5年以上居住する親がいる新婚世帯）2 区内転居助成（区内に引き続き1年以上居住している子育て世帯）'},
    {'name': '居住安定支援家賃助成',
     'detail': '千代田区内に引き続き2年以上居住する高齢者世帯・障害者世帯・ひとり親世帯が、取り壊し等により転居を余儀なくされた場合や、やむを得ない事由により世帯の所得が著しく減少した場合、または安全上・衛生上劣悪な状態の住宅に居住している場合に、家賃等の一部を助成'},
    {'name': '高齢者等民間賃貸住宅入居支援制度',
     'detail': '千代田区内に居住する高齢者・障害者・ひとり親世帯のいずれかが区内での転居をする際に、保証会社利用料への助成等を行う'},
    {'name': '高齢者等安心居住支援家賃助成制度',
     'detail': '千代田区内の持ち家に居住する方で、その居住する住宅のバリアフリー改修が困難等の理由により居宅での日常生活に支障が生じ、緊急に代替となる住宅の確保が必要と認められる要介護高齢者世帯等に対して、民間賃貸住宅の家賃等の一部を助成（月額5万円、２年間）'}]},
  {'title': 'その他独自の取り組み', 'items': [{'name': '', 'detail': 'なし'}]}],
 'character': 'chiyoda.jpg',
 'copyright': '©千代田区(仮)'}